In [2]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LassoCV, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from xgboost import XGBClassifier, XGBRegressor

In [3]:
import os
os.system("pip install doubleml")
import doubleml as dml
from doubleml.datasets import fetch_401K

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
# data reading and binarization
dataset = pd.read_csv('/content/drive/MyDrive/stat274/IVdata-1.csv')
dataset['instrument'] = (dataset.xunshi).apply(int)
dataset['treatment'] = (dataset.tenure >= 3).apply(int)
dataset['outcome'] = (dataset.xc_lockdown).apply(int)

In [6]:
# Basic model
data_dml_base_iv = dml.DoubleMLData(dataset,
                                    y_col='outcome',
                                    d_cols='treatment',
                                    z_cols='instrument',
                                    x_cols=['mayor_age'])

In [8]:
# Random Forest
randomForest = RandomForestRegressor(n_estimators=500)
randomForest_class = RandomForestClassifier(n_estimators=500)

np.random.seed(123)
dml_iivm_forest = dml.DoubleMLIIVM(data_dml_base_iv,
                                   ml_g = randomForest,
                                   ml_m = randomForest_class,
                                   ml_r = randomForest_class,
                                   subgroups = {'always_takers': False,
                                                'never_takers': True},
                                   trimming_threshold = 0.01,
                                   n_folds = 3)

# Set nuisance-part specific parameters
dml_iivm_forest.set_ml_nuisance_params('ml_g0', 'treatment', {
    'max_depth': 6, 'max_features': 1, 'min_samples_leaf': 7})
dml_iivm_forest.set_ml_nuisance_params('ml_g1', 'treatment', {
    'max_depth': 6, 'max_features': 1, 'min_samples_leaf': 5})
dml_iivm_forest.set_ml_nuisance_params('ml_r1', 'treatment', {
    'max_depth': 4, 'max_features': 1, 'min_samples_leaf': 6})

dml_iivm_forest.fit(store_predictions=True) 
forest_summary = dml_iivm_forest.summary

forest_summary

,coef,std err,t,P>|t|,2.5 %,97.5 %
treatment,0.141952,0.304688,0.465892,0.641293,-0.455226,0.739129
